In [15]:
import numpy as np
import random
from copy import deepcopy

In [16]:
COLOUR_DICT = {
    0: "One", 1: "Blue", 2: "Dark Blue", 3: "Light Blue", 4: "Red", 5: "Orange"
}

In [17]:
# Maybe we can define the grid as an array of tuples, 1st argument indicating whether it is filled, and the 2nd, indicating the type of colour
INITIAL_GRID = [
    [[0, 1], [0, 5], [0, 4], [0, 2], [0, 3]],
    [[0, 3], [0, 1], [0, 5], [0, 4], [0, 2]],
    [[0, 2], [0, 3], [0, 1], [0, 5], [0, 4]],
    [[0, 4], [0, 2], [0, 3], [0, 1], [0, 5]],
    [[0, 5], [0, 4], [0, 2], [0, 3], [0, 1]]
]

# We can also define the tiles that are filled to place gems in the grid, zeros are used here to indicate that nothing is currently present
INITIAL_TILES = [
    [0], [0,0], [0,0,0], [0,0,0,0], [0,0,0,0,0]
]

In [18]:
# Next, let's make a generator to randomly produce assortments of the tops within the game
# That is, we want to produce 7 4-tuples with varying distributions of colour and this MUST be randomized (There are 100 tiles in total and 20 of each colour)
COLOUR_DICT

{0: 'One', 1: 'Blue', 2: 'Dark Blue', 3: 'Light Blue', 4: 'Red', 5: 'Orange'}

In [19]:
def generate_distributions():
    
    while True: 
        curr_list = []
        for i in range(7):
            assort_tuple = (random.randint(1,5), random.randint(1,5), random.randint(1,5), random.randint(1,5))
            curr_list.append(assort_tuple)
        
        if verify_distribution(curr_list):
            return curr_list

def verify_distribution(nums):
    curr_dict = {}
    for tups in nums:
        for x in tups:
            if x in curr_dict:
                curr_dict[x] += 1
            else:
                curr_dict[x] = 1

    for x in curr_dict:
        if curr_dict[x] > 20:
            return False
        
    return True

def convert_nums_to_colours(nums):
    new_list = []
    for x,y,z,w in nums:
        new_list.append((COLOUR_DICT[x], COLOUR_DICT[y], COLOUR_DICT[x], COLOUR_DICT[w]))
    
    return new_list

In [20]:
random_dist = generate_distributions()
random_dist_colors = convert_nums_to_colours(random_dist)

random_dist_colors

[('Red', 'Dark Blue', 'Red', 'Orange'),
 ('Red', 'Dark Blue', 'Red', 'Red'),
 ('Red', 'Light Blue', 'Red', 'Light Blue'),
 ('Dark Blue', 'Red', 'Dark Blue', 'Red'),
 ('Blue', 'Light Blue', 'Blue', 'Red'),
 ('Red', 'Light Blue', 'Red', 'Orange'),
 ('Dark Blue', 'Red', 'Dark Blue', 'Dark Blue')]

In [21]:
# Remember here that we have to pick A COLOUR amongst the tiles which are present on the top, and the rest of the colors are shoved within the middle
# Need to keep a track of those colours in the middle

In [22]:
# 0 represents the ONE tile
MIDDLE_TILES = [0]

In [23]:
# For predictions, we want an algorithm that will tell the user which of the 7 tiles are best, and of those colours within the tiles, which should be selected.
# Should we simulate choosing each of the tiles and test for the possible points which can be obtained and see which selection maxes the points? 
# This would require simulations of the game
# Let's make a player class for this to represent the information that each player will have

In [24]:
class Player:
    def __init__(self):
        self.grid = deepcopy(INITIAL_GRID)
        self.tiles = deepcopy(INITIAL_TILES)
        self.score = 0
        self.punishment = 0
    
    # This function tells the user which row is best to fill out when given a certain number of tiles
    def best_tile_row(self, tiles):
        num_tiles = len(tiles)
        tile_type = tiles[0]
        app_rows = []
        
        # Begin by getting the rows of which that match how many tiles are available to be slotted in (Check that tiles also have to be of the same kind)
        for row in self.tiles:
            num_empty = 0
            for space in row:
                row_tile_type = -1
                if space == 0:
                    num_empty += 1
                else:
                    row_tile_type = space 
            
            if num_empty == num_tiles and (row_tile_type == -1 or row_tile_type == tile_type):
                app_rows.append(row)

        # We can calculate which of the rows is better suited in obtaining a higher score
        scores = []
        for row in app_rows:
            row_index = len(row) - 1
            test_grid = deepcopy(self.grid)
            for i in range(5):
                filled = test_grid[row_index][i][0]
                tiletype = test_grid[row_index][i][1]
                if not filled and tiletype == tile_type:
                    test_grid[row_index][i][0] = 1
                    score = self.get_score(test_grid, row_index, i)
                    scores.append(score)
        
        # If there was a row to be completed then...
        if scores:
            # Get the index of the highest score
            index_highest = scores.index(max(scores))
    
            best_row = app_rows[index_highest]
            
            # Returns the best score achievable with the row to be filled 
            return max(scores), best_row
        
            #row_tiles_index = len(best_row) - 1
            # self.tiles[row_tiles_index] = [tile_type for i in range(len(best_row))] # Place down the tiles within the tiles array
            
        else:
            # Perform action that calculates where else best to select those tiles
            
            # First instinct should perhaps be filling out the rows that have yet to be completed (only partially)
            # Or insert into the empty rows as a final resort
            
            non_empty_rows = []
            empty_rows = []
            for row in self.tiles:
                if not all(elem == 0 for elem in row) and row[0] == tile_type:
                    non_empty_rows.append(row)
                else:
                    empty_rows.append(row)
            
            if non_empty_rows:
                return 0, non_empty_rows[0]
            else:
                return 0, empty_rows[0]
            
            
    # The row and column is to tell where the most recent tile has been added so the score can be calculated accordingly
    def get_score(self, grid, row, column):
        
        # Let us check column wise and row wise, how the score will be calculated
        # We go from the tile itself, to the left and check how many consecutively and then to the right consecutively and minus one, same goes for column
        
        score_row = grid[row]
        left_score = 0
        right_score = 0
        for i in range(column-1, -1, -1):
            if not score_row[i][0]:
                break
            left_score += 1
                
        for i in range(column+1, 5):
            if not score_row[i][0]:
                break
            right_score += 1
        
        row_score = left_score + right_score
        
        up_score = 0
        down_score = 0
        score_column = [r[column] for r in grid]
        
        for i in range(row-1, -1, -1):
            if not score_column[i][0]:
                break
            up_score += 1
                
        for i in range(row+1, 5):
            if not score_column[i][0]:
                break
            down_score += 1
        
        column_score = up_score + down_score
        
        total_score = row_score + column_score
        if total_score == 0:
            total_score = 1
            
        return total_score
    
    # This function does not make any changes to the tops nor the number of tiles themselves, it only returns the chosen number of tiles on the selected top
    # The numbers will only be changed after the best possibly outcome is calculated and selected
    def take_colors(self, middles, tiles_arr, tile_colour, TAKE_MIDDLES = False):
        if not TAKE_MIDDLES:
            chosen_tiles = [tile for tile in tiles_arr if tile == tile_colour]
            return chosen_tiles
        
        else:
            chosen_tiles = [tile for tile in middles if tile == tile_colour]
            return chosen_tiles
            
        
        

In [25]:
# Generate the random starting distribution of tiles and begi

test_player1 = Player()
test_player2 = Player()
test_player3 = Player()
player_arr = [test_player1, test_player2, test_player3]
starting_tiles = generate_distributions()
middle_tiles = [0]

In [26]:
# With the appropriate functions we can try to simulate what a round of azul might look like, we can take an approach that maximizes our points and minimizing the enemy's points
# Every time a circle of turns pass, we can update the number of tiles left available and repeat the calculations, since it is not set in stone for what the others might do

In [28]:
# Lets perform a test run calculation

def generate_options(tiles, middle_tiles, player):
    options_arr = []
    unique_middle_colors = list(set(middle_tiles))
    if len(unique_middle_colors) > 1:
        for color in unique_middle_colors:
            chosen = player.take_colors(middle_tiles, [], color, TAKE_MIDDLES=True)
            
            # Should minus one here
            points_earned, best_row = player.best_tile_row(chosen)
            score_tuple = (points_earned, best_row, color, chosen)

            options_arr.append(score_tuple)
    
    for top in tiles:
        unique_colors = list(set(top))

        for color in unique_colors:
            chosen = player.take_colors(middle_tiles, top, color, TAKE_MIDDLES=False)
        
            # For these chosen tiles, now fill in the tiles and get the expected score
            points_earned, best_row = player.best_tile_row(chosen)
            score_tuple = (points_earned, best_row, color, top)

            options_arr.append(score_tuple)
        
    
    return options_arr
    
options_arr = generate_options(starting_tiles, middle_tiles, test_player1)
options_arr

[(1, [0], 1, (3, 4, 5, 1)),
 (1, [0], 3, (3, 4, 5, 1)),
 (1, [0], 4, (3, 4, 5, 1)),
 (1, [0], 5, (3, 4, 5, 1)),
 (1, [0, 0, 0], 2, (2, 2, 2, 4)),
 (1, [0], 4, (2, 2, 2, 4)),
 (1, [0], 1, (1, 2, 2, 2)),
 (1, [0, 0, 0], 2, (1, 2, 2, 2)),
 (1, [0, 0], 1, (1, 5, 1, 5)),
 (1, [0, 0], 5, (1, 5, 1, 5)),
 (1, [0], 1, (4, 1, 5, 4)),
 (1, [0, 0], 4, (4, 1, 5, 4)),
 (1, [0], 5, (4, 1, 5, 4)),
 (1, [0], 2, (4, 3, 3, 2)),
 (1, [0, 0], 3, (4, 3, 3, 2)),
 (1, [0], 4, (4, 3, 3, 2)),
 (1, [0, 0, 0, 0], 1, (1, 1, 1, 1))]

In [ ]:
# Now we can compute a tree of possibilities
# Go through the cycle of players until each possibility has been explored

curr_player = 0 # Increase this and mod len of players when we cycle
middle = [0]

for option in options_arr:
    tops_copy = deepcopy(starting_tiles)
    print(option)
    # Let's look at the possibility for a current option
    score, row, color, top = option
    
    # From the color selected and the top chosen, we remove it from the list of tops, push the remaining into the middle
    leftover = [tile for tile in top if tile != color]
    middle += leftover
    
    tops_copy.remove(top)
    
    # Now we have a
    
